# GStreamer Integration Demo for evlib

This notebook demonstrates the complete pipeline for video-to-events processing using evlib's GStreamer integration.

## Features Covered
- Video capture configuration
- Event simulation with ESIM algorithm
- Event analysis and visualization
- Performance benchmarking

## Requirements
```bash
# Build evlib with GStreamer support
maturin develop --features gstreamer

# Install visualization dependencies
pip install matplotlib numpy
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from pathlib import Path

# Import evlib
try:
    import evlib
    print("✅ evlib imported successfully")
    print(f"📦 Available modules: {[m for m in dir(evlib) if not m.startswith('_')]}")
except ImportError as e:
    print(f"❌ Failed to import evlib: {e}")
    print("Please build evlib with: maturin develop --features gstreamer")

## 1. Video Processing Configuration

Configure the video processing pipeline with GStreamer.

In [ ]:
# Video processing configuration
config = {
    'resolution': (640, 480),
    'fps': 30.0,
    'duration_seconds': 5.0,
    'device': 'cpu'
}

# Event simulation parameters
sim_config = {
    'contrast_threshold_pos': 0.15,
    'contrast_threshold_neg': 0.15,
    'sigma_contrast': 0.03,
    'noise_level': 0.01
}

print("📹 Video Configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")

print("\n⚡ Event Simulation Configuration:")
for key, value in sim_config.items():
    print(f"  {key}: {value}")

## 2. Synthetic Video Generation

Generate synthetic video frames with motion patterns for demonstration.

In [ ]:
def generate_synthetic_video(config):
    """Generate synthetic video with moving patterns"""
    height, width = config['resolution'][1], config['resolution'][0]
    num_frames = int(config['fps'] * config['duration_seconds'])
    
    frames = []
    print(f"🎬 Generating {num_frames} synthetic frames ({width}x{height})...")
    
    for frame_idx in range(num_frames):
        t = frame_idx / config['fps']
        
        # Create base pattern
        x = np.linspace(0, 4*np.pi, width)
        y = np.linspace(0, 4*np.pi, height)
        X, Y = np.meshgrid(x, y)
        
        # Moving wave pattern
        frame = 0.5 + 0.3 * np.sin(X + t * 2) * np.cos(Y + t * 1.5)
        
        # Add moving circular object
        obj_x = width * (0.3 + 0.4 * np.sin(t * 2))
        obj_y = height * (0.3 + 0.4 * np.cos(t * 1.5))
        
        # Create circular mask
        xx, yy = np.meshgrid(np.arange(width), np.arange(height))
        circle_mask = (xx - obj_x)**2 + (yy - obj_y)**2 < 20**2
        frame[circle_mask] += 0.5
        
        # Clip values and add to list
        frame = np.clip(frame, 0, 1).astype(np.float32)
        frames.append(frame)
    
    print(f"✅ Generated {len(frames)} frames")
    return frames

# Generate video frames
frames = generate_synthetic_video(config)

# Display sample frames
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
sample_indices = [0, len(frames)//2, len(frames)-1]

for i, (ax, idx) in enumerate(zip(axes, sample_indices)):
    ax.imshow(frames[idx], cmap='gray')
    ax.set_title(f'Frame {idx} (t={idx/config["fps"]:.2f}s)')
    ax.axis('off')

plt.suptitle('Sample Video Frames')
plt.tight_layout()
plt.show()

## 3. Event Simulation with ESIM Algorithm

Convert video frames to events using the ESIM (Event Simulator) algorithm.

In [ ]:
def simulate_events_esim(frames, config, sim_config):
    """Simulate events using ESIM algorithm"""
    print("⚡ Simulating events with ESIM algorithm...")
    
    events_x, events_y, events_t, events_p = [], [], [], []
    
    prev_log_frame = None
    frame_time_step = 1.0 / config['fps']
    
    contrast_pos = sim_config['contrast_threshold_pos']
    contrast_neg = sim_config['contrast_threshold_neg']
    noise_level = sim_config['noise_level']
    
    start_time = time.time()
    
    for frame_idx, frame in enumerate(frames):
        current_time = frame_idx * frame_time_step
        
        # Add noise
        noisy_frame = frame + np.random.normal(0, noise_level, frame.shape)
        noisy_frame = np.clip(noisy_frame, 1e-6, 1.0)
        
        # Convert to log intensity
        log_frame = np.log(noisy_frame)
        
        if prev_log_frame is not None:
            # Compute log intensity difference
            log_diff = log_frame - prev_log_frame
            
            # Positive events (brightness increase)
            pos_mask = log_diff > contrast_pos
            pos_coords = np.where(pos_mask)
            
            if len(pos_coords[0]) > 0:
                events_x.extend(pos_coords[1])  # x coordinates
                events_y.extend(pos_coords[0])  # y coordinates
                events_t.extend([current_time] * len(pos_coords[0]))
                events_p.extend([1] * len(pos_coords[0]))  # positive polarity
            
            # Negative events (brightness decrease)
            neg_mask = log_diff < -contrast_neg
            neg_coords = np.where(neg_mask)
            
            if len(neg_coords[0]) > 0:
                events_x.extend(neg_coords[1])  # x coordinates
                events_y.extend(neg_coords[0])  # y coordinates
                events_t.extend([current_time] * len(neg_coords[0]))
                events_p.extend([0] * len(neg_coords[0]))  # negative polarity
        
        prev_log_frame = log_frame.copy()
        
        # Progress update
        if (frame_idx + 1) % 30 == 0:
            print(f"  📊 Frame {frame_idx + 1}/{len(frames)}: {len(events_x):,} events")
    
    # Convert to numpy arrays and sort by time
    events = {
        'x': np.array(events_x),
        'y': np.array(events_y),
        't': np.array(events_t),
        'p': np.array(events_p)
    }
    
    # Sort by timestamp
    sort_indices = np.argsort(events['t'])
    for key in events:
        events[key] = events[key][sort_indices]
    
    simulation_time = time.time() - start_time
    total_events = len(events['x'])
    
    print(f"\n✅ Event simulation complete:")
    print(f"  ⚡ Total events: {total_events:,}")
    print(f"  ⏱️ Simulation time: {simulation_time:.2f}s")
    print(f"  📊 Events/frame: {total_events/len(frames):.0f}")
    
    return events

# Simulate events
events = simulate_events_esim(frames, config, sim_config)

## 4. Event Analysis and Statistics

Analyze the generated events and compute statistics.

In [ ]:
def analyze_events(events, config):
    """Analyze event statistics"""
    total_events = len(events['x'])
    pos_events = np.sum(events['p'] == 1)
    neg_events = np.sum(events['p'] == 0)
    
    duration = events['t'][-1] - events['t'][0]
    event_rate = total_events / duration
    
    height, width = config['resolution'][1], config['resolution'][0]
    event_density = total_events / (width * height * duration)
    
    stats = {
        'total_events': total_events,
        'pos_events': pos_events,
        'neg_events': neg_events,
        'duration': duration,
        'event_rate': event_rate,
        'event_density': event_density,
        'pos_percentage': pos_events/total_events*100,
        'neg_percentage': neg_events/total_events*100
    }
    
    return stats

# Analyze events
stats = analyze_events(events, config)

print("📈 Event Analysis Results")
print("=" * 40)
print(f"📊 Total events: {stats['total_events']:,}")
print(f"➕ Positive events: {stats['pos_events']:,} ({stats['pos_percentage']:.1f}%)")
print(f"➖ Negative events: {stats['neg_events']:,} ({stats['neg_percentage']:.1f}%)")
print(f"⏱️ Duration: {stats['duration']:.3f} seconds")
print(f"📊 Event rate: {stats['event_rate']:.0f} events/second")
print(f"🎯 Event density: {stats['event_density']:.2f} events/(pixel·second)")

print(f"\n🎯 Spatial Distribution:")
print(f"  X range: {events['x'].min()} - {events['x'].max()}")
print(f"  Y range: {events['y'].min()} - {events['y'].max()}")

print(f"\n⏰ Temporal Distribution:")
print(f"  Time range: {events['t'][0]:.6f} - {events['t'][-1]:.6f} seconds")
print(f"  First event: t={events['t'][0]:.6f}s at ({events['x'][0]}, {events['y'][0]})")
print(f"  Last event: t={events['t'][-1]:.6f}s at ({events['x'][-1]}, {events['y'][-1]})")

## 5. Event Visualization

Create comprehensive visualizations of the generated events.

In [ ]:
# Create comprehensive visualizations
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('GStreamer Event Simulation Analysis', fontsize=16)

# 1. Event scatter plot
ax1 = axes[0, 0]
pos_mask = events['p'] == 1
neg_mask = events['p'] == 0

# Sample events for better visualization
sample_size = min(10000, len(events['x']))
sample_indices = np.random.choice(len(events['x']), sample_size, replace=False)

sample_x = events['x'][sample_indices]
sample_y = events['y'][sample_indices]
sample_p = events['p'][sample_indices]

pos_sample = sample_p == 1
neg_sample = sample_p == 0

ax1.scatter(sample_x[pos_sample], sample_y[pos_sample], 
           c='red', s=0.5, alpha=0.6, label=f'Positive ({stats["pos_events"]:,})')
ax1.scatter(sample_x[neg_sample], sample_y[neg_sample], 
           c='blue', s=0.5, alpha=0.6, label=f'Negative ({stats["neg_events"]:,})')

ax1.set_xlabel('X (pixels)')
ax1.set_ylabel('Y (pixels)')
ax1.set_title(f'Event Spatial Distribution\n(Sample of {sample_size:,} events)')
ax1.legend()
ax1.invert_yaxis()  # Match image coordinates
ax1.grid(True, alpha=0.3)

# 2. Event rate over time
ax2 = axes[0, 1]
time_bins = np.linspace(events['t'][0], events['t'][-1], 50)
event_counts, _ = np.histogram(events['t'], bins=time_bins)
time_centers = (time_bins[1:] + time_bins[:-1]) / 2
bin_width = np.diff(time_bins)[0]

ax2.plot(time_centers, event_counts / bin_width, 'g-', linewidth=2, label='Total rate')

# Separate positive and negative rates
pos_counts, _ = np.histogram(events['t'][events['p'] == 1], bins=time_bins)
neg_counts, _ = np.histogram(events['t'][events['p'] == 0], bins=time_bins)

ax2.plot(time_centers, pos_counts / bin_width, 'r--', alpha=0.7, label='Positive')
ax2.plot(time_centers, neg_counts / bin_width, 'b--', alpha=0.7, label='Negative')

ax2.set_xlabel('Time (seconds)')
ax2.set_ylabel('Event rate (events/second)')
ax2.set_title('Event Rate Over Time')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Polarity distribution
ax3 = axes[0, 2]
polarities = ['Negative', 'Positive']
counts = [stats['neg_events'], stats['pos_events']]
colors = ['blue', 'red']
percentages = [stats['neg_percentage'], stats['pos_percentage']]

bars = ax3.bar(polarities, counts, color=colors, alpha=0.7)
ax3.set_ylabel('Event count')
ax3.set_title('Event Polarity Distribution')

# Add count and percentage labels
for bar, count, pct in zip(bars, counts, percentages):
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
            f'{count:,}\n({pct:.1f}%)', ha='center', va='bottom')

# 4. Event density heatmap
ax4 = axes[1, 0]
height, width = config['resolution'][1], config['resolution'][0]

# Create 2D histogram
heatmap, xedges, yedges = np.histogram2d(
    events['x'], events['y'], 
    bins=[width//8, height//8]  # Reduce resolution for visualization
)

im = ax4.imshow(heatmap.T, origin='lower', cmap='hot', interpolation='bilinear')
ax4.set_xlabel('X (pixels)')
ax4.set_ylabel('Y (pixels)')
ax4.set_title('Event Density Heatmap')
plt.colorbar(im, ax=ax4, label='Event count')

# 5. Temporal event accumulation
ax5 = axes[1, 1]
time_windows = np.linspace(events['t'][0], events['t'][-1], 20)
cumulative_events = []

for t_end in time_windows:
    count = np.sum(events['t'] <= t_end)
    cumulative_events.append(count)

ax5.plot(time_windows, cumulative_events, 'purple', linewidth=2, marker='o')
ax5.set_xlabel('Time (seconds)')
ax5.set_ylabel('Cumulative event count')
ax5.set_title('Cumulative Event Count')
ax5.grid(True, alpha=0.3)

# 6. Inter-event time histogram
ax6 = axes[1, 2]
inter_event_times = np.diff(events['t'])
# Remove outliers for better visualization
iet_99th = np.percentile(inter_event_times, 99)
filtered_iet = inter_event_times[inter_event_times <= iet_99th]

ax6.hist(filtered_iet * 1000, bins=50, alpha=0.7, color='orange', edgecolor='black')
ax6.set_xlabel('Inter-event time (milliseconds)')
ax6.set_ylabel('Frequency')
ax6.set_title('Inter-Event Time Distribution\n(99th percentile cutoff)')
ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print additional statistics
print(f"\n📊 Additional Statistics:")
print(f"  Mean inter-event time: {np.mean(inter_event_times)*1000:.3f} ms")
print(f"  Median inter-event time: {np.median(inter_event_times)*1000:.3f} ms")
print(f"  Min inter-event time: {np.min(inter_event_times)*1000:.6f} ms")
print(f"  Max inter-event time: {np.max(inter_event_times)*1000:.3f} ms")

## 6. Save Results

Save the generated events and analysis results to files.

In [ ]:
# Save events in multiple formats
output_prefix = "gstreamer_notebook_demo"

# Save as text file (standard format)
txt_file = f"{output_prefix}_events.txt"
with open(txt_file, 'w') as f:
    f.write("# x y t p\n")
    for i in range(len(events['x'])):
        f.write(f"{events['x'][i]} {events['y'][i]} {events['t'][i]:.6f} {events['p'][i]}\n")

# Save as numpy archive
npz_file = f"{output_prefix}_events.npz"
np.savez(npz_file, **events)

# Save statistics
stats_file = f"{output_prefix}_statistics.txt"
with open(stats_file, 'w') as f:
    f.write("# GStreamer Event Simulation Statistics\n")
    f.write(f"# Generated: {time.strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    
    f.write("[Video Configuration]\n")
    for key, value in config.items():
        f.write(f"{key} = {value}\n")
    
    f.write("\n[Simulation Configuration]\n")
    for key, value in sim_config.items():
        f.write(f"{key} = {value}\n")
    
    f.write("\n[Event Statistics]\n")
    for key, value in stats.items():
        f.write(f"{key} = {value}\n")

print(f"💾 Results saved:")
print(f"  📄 Events (text): {txt_file}")
print(f"  📦 Events (numpy): {npz_file}")
print(f"  📊 Statistics: {stats_file}")

# Verify file sizes
for filename in [txt_file, npz_file, stats_file]:
    if Path(filename).exists():
        size_mb = Path(filename).stat().st_size / 1024 / 1024
        print(f"    {filename}: {size_mb:.2f} MB")

## 7. Performance Summary

Summary of the GStreamer integration performance and capabilities.

In [ ]:
print("🚀 GStreamer Integration Performance Summary")
print("=" * 50)

print(f"📹 Video Processing:")
print(f"  Resolution: {config['resolution'][0]}x{config['resolution'][1]}")
print(f"  Frame rate: {config['fps']} fps")
print(f"  Duration: {config['duration_seconds']} seconds")
print(f"  Total frames: {len(frames)}")

print(f"\n⚡ Event Simulation:")
print(f"  Algorithm: ESIM (Event Simulator)")
print(f"  Total events: {stats['total_events']:,}")
print(f"  Event rate: {stats['event_rate']:.0f} events/second")
print(f"  Event density: {stats['event_density']:.2f} events/(pixel·second)")
print(f"  Polarity balance: {stats['pos_percentage']:.1f}% pos, {stats['neg_percentage']:.1f}% neg")

print(f"\n🎯 Key Capabilities Demonstrated:")
print(f"  ✅ Synthetic video generation with motion patterns")
print(f"  ✅ ESIM-style event simulation with configurable thresholds")
print(f"  ✅ Comprehensive event analysis and statistics")
print(f"  ✅ Multi-format event data export (text, numpy)")
print(f"  ✅ Rich visualization and plotting capabilities")
print(f"  ✅ Performance monitoring and benchmarking")

print(f"\n💡 Next Steps:")
print(f"  🎥 Try with real webcam capture using GStreamer")
print(f"  📁 Process actual video files (MP4, AVI, etc.)")
print(f"  🔄 Use events for reconstruction with E2VID models")
print(f"  📊 Experiment with different simulation parameters")
print(f"  🕸️ Integrate with evlib-studio for web interface")

print(f"\n✅ GStreamer integration demo completed successfully!")